In [1]:
import eec
import numpy as np
import gensim
import pandas as pd

In [2]:
keyed_vectors = gensim.models.KeyedVectors.load('./data/word2vec.model')

In [3]:
keyed_vectors.wv.most_similar('lettuce')

[('romaine', 0.8329761028289795),
 ('head', 0.7326541543006897),
 ('boston', 0.7135635614395142),
 ('cabbage', 0.7096205353736877),
 ('rack', 0.6752064228057861),
 ('spring', 0.6698594093322754),
 ('turnip', 0.6696361303329468),
 ('pig', 0.6667503714561462),
 ('cup', 0.6314897537231445),
 ('kaffir', 0.6175163388252258)]

In [4]:
entity_repository = eec.BaseEntityRepository(
    entities= [],
    keyed_vectors=keyed_vectors,
    last_id=0   
)

In [5]:
food_com_df = pd.read_csv('./data/all.csv')
food_com_df

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,phrase,target,id_phrase
0,0,0,0,0,0,4308,"medium heads bibb or red leaf lettuce, washed,...",lettuce,0
1,1,1,1,1,1,4308,mixed baby lettuces and spring greens,lettuce,1
2,2,2,2,2,2,4308,romaine lettuce leaf,lettuce,2
3,3,3,3,3,3,4308,iceberg lettuce leaf,lettuce,3
4,4,4,4,4,4,4308,red romaine lettuce,lettuce,4
...,...,...,...,...,...,...,...,...,...
11654,11654,11654,11654,11654,11654,6702,soybeans,soybean,11654
11655,11655,11655,11655,11655,11655,3318,goose,goose,11655
11656,11656,11656,11656,11656,11656,47,ajwain,ajwain,11656
11657,11657,11657,11657,11657,11657,750,brinjals,brinjal,11657


In [6]:
food_com_df.to_csv('./data/all.csv')

In [7]:
entities : list[eec.BaseEntity] = []
for index, row in food_com_df.iterrows():
    entities.append(eec.BaseEntity(
        entity_id=str(index),
        entity_source='food_com',
        entity_source_id=str(index),
        mention=row['phrase'],
    ))

entity_repository.add_entities(entities)

[1-medium heads bibb or red leaf lettuce, washed, dried* and torn into pieces,
 2-mixed baby lettuces and spring greens,
 3-romaine lettuce leaf,
 4-iceberg lettuce leaf,
 5-red romaine lettuce,
 6-head romaine lettuce,
 7-curly endive lettuce,
 8-romaine lettuce hearts,
 9-baby leaf lettuce,
 10-head of lettuce,
 11-head iceberg lettuce,
 12-boston lettuce leaf,
 13-heads of lettuce,
 14-green leaf lettuce,
 15-romaine lettuce leaves,
 16-iceberg lettuce leaves,
 17-butter lettuce leaf,
 18-boston lettuce leaves,
 19-red leaf lettuce,
 20-butter lettuce leaves,
 21-baby lettuces,
 22-red-leaf lettuce,
 23-butterhead lettuce,
 24-lambs lettuce,
 25-lettuce head,
 26-lettuce cup,
 27-lettuce leaves,
 28-boston lettuce,
 29-butter lettuce,
 30-romaine lettuce,
 31-bibb lettuce,
 32-iceberg lettuce,
 33-lettuce leaf,
 34-head lettuce,
 35-gem lettuce,
 36-lettuce cups,
 37-lettuce greens,
 38-cos lettuce,
 39-leaf lettuce,
 40-frisee lettuce,
 41-chicory lettuce,
 42-lettuce,
 43-sugar fr

In [8]:
cluster_repository = eec.BaseClusterRepository(
    entity_repository=entity_repository,
    clusters=[],
)

In [9]:
mention_clustering_method = eec.BaseMentionClusteringMethod(
    name='mention_clustering_method',
    entity_repository=entity_repository,
    cluster_repository=cluster_repository,
)

In [10]:
eec.EntityClustererBridge().set_cluster_repository(cluster_repository)
eec.EntityClustererBridge().set_entity_repository(entity_repository)
eec.EntityClustererBridge().set_mention_clustering_method(mention_clustering_method)

In [11]:
entity = eec.EntityClustererBridge().entity_repository.get_random_unlabeled_entity()

In [12]:
entity

30-romaine lettuce

In [13]:
eec.EntityClustererBridge().mention_clustering_method.getPossibleClusters(entity)

No cluster vectors found Fallback to only string similarity
Fallback failed! Create new cluster!


[]

In [22]:
cluster_repository.add_cluster(eec.BaseCluster(
    cluster_id='2',
    cluster_name='dip sauce',
    entities=[],
))

Cluster 1-dip sauce

In [23]:
cluster_repository.add_entity_to_cluster('2', entity2.entity_id)

NotFoundException: Cluster with id {cluster_id} not found.

In [19]:
entity2 = eec.EntityClustererBridge().entity_repository.get_random_unlabeled_entity()

In [20]:
entity2

8883-spinach dip

In [21]:
eec.EntityClustererBridge().mention_clustering_method.getPossibleClusters(entity2)

[Cluster 0-cabbage]